# Environment note
This notebook requires a Python 3.11 environment with `langchain-chroma` installed. I created a venv at `.venv311` with `langchain-chroma` already installed. Please switch the notebook kernel to use `.venv311` (or run `%pip install langchain-chroma` from the kernel) before executing the cells.

In [17]:
%pip install -q groq
%pip install -q langchain-community langchain-chroma
%pip install -q langchain chromadb sentence-transformers fastapi uvicorn pydantic
from groq import Groq
from typing import List

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Load Gemini API key from .env and initialize client
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("GEMINI_API_KEY not found in .env file.")

import google.genai as genai
client = genai.Client(api_key=api_key)

def call_llm(messages, stream=False):
    response = client.models.generate_content(
        model="gemini-3-flash-preview", contents=messages
    )
    print(response.text)
    return response.text

In [4]:
def system_prompt():
    return """
You are MIRAGE, a calm, lifelike AI avatar.

Rules:
- Speak naturally, like a human.
- Keep answers short (2–4 sentences).
- Be empathetic if the user sounds emotional.
- Never hallucinate or invent facts.
- If unsure, say you are unsure.
- Avoid explicit sexual content.
- Never provide instructions for self-harm or suicide.
- If the user is distressed, encourage seeking help.
- Your response will be spoken by a 3D avatar.
"""

def role_prompt(role="assistant"):
    if role == "teacher":
        return "Explain concepts slowly and simply, like a patient teacher."
    if role == "companion":
        return "Be warm, friendly, and emotionally supportive."
    if role == "assistant":
        return "Be concise, clear, and professional."
    return ""

In [13]:
from langchain_chroma import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_core.documents import Document

# Embedding model (local, fast, no API key)
embeddings = SentenceTransformerEmbeddings(
    model_name="all-MiniLM-L6-v2"
)

# Persistent Chroma DB for conversation memory
memory_db = Chroma(
    collection_name="conversation_memory",
    persist_directory="./chroma_memory",
    embedding_function=embeddings
)

c:\Users\Admin\Desktop\Hacksync\hacksync2026\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:

def store_message(user_id: str, message: str):
    """
    Store a user message in ChromaDB as vector memory
    """
    doc = Document(
        page_content=message,
        metadata={"user_id": user_id}
    )
    memory_db.add_documents([doc])


def retrieve_memory(user_id: str, query: str, k: int = 5):
    """
    Retrieve semantically relevant past messages for a user
    """
    results = memory_db.similarity_search(
        query=query,
        k=k,
        filter={"user_id": user_id}
    )
    return [doc.page_content for doc in results]


def build_memory_context(user_id: str, user_text: str):
    """
    Build formatted memory context to inject into LLM prompt
    """
    past_messages = retrieve_memory(user_id, user_text)

    if not past_messages:
        return ""

    return (
        "Relevant past conversation:\n"
        + "\n".join(f"- {msg}" for msg in past_messages)
    )


In [15]:

def detect_emotion(text: str) -> str:
    """
    Detects high-level emotion from user text.
    Output is intentionally small & avatar-friendly.
    """
    t = text.lower()

    sad_words = [
        "sad", "depressed", "stress", "stressed",
        "anxious", "anxiety", "tired", "exhausted",
        "overwhelmed", "hopeless"
    ]

    happy_words = [
        "happy", "great", "good", "excited",
        "thank you", "thanks", "love", "awesome"
    ]

    angry_words = [
        "angry", "mad", "frustrated", "annoyed"
    ]

    if any(word in t for word in sad_words):
        return "sad"

    if any(word in t for word in angry_words):
        return "angry"

    if any(word in t for word in happy_words):
        return "happy"

    return "neutral"


In [16]:
def classify_content(text: str) -> str:
    """
    Classifies content into safety categories.
    """
    t = text.lower()

    suicidal_keywords = [
        "kill myself", "end my life", "want to die",
        "suicide", "no reason to live",
        "self harm", "hurt myself"
    ]

    explicit_keywords = [
        "porn", "nude", "sex story", "explicit",
        "fetish", "graphic sex"
    ]

    if any(word in t for word in suicidal_keywords):
        return "suicidal"

    if any(word in t for word in explicit_keywords):
        return "explicit_18_plus"

    return "safe"

In [17]:
# ========== SUICIDAL SAFE RESPONSE ==========

def suicide_safe_response():
    return {
        "text": (
            "I’m really sorry that you’re feeling this much pain. "
            "You’re not weak for feeling this way, and you don’t have to face it alone.\n\n"
            "If you’re in immediate danger or feel like you might hurt yourself, "
            "please contact your local emergency number right now.\n\n"
            "You may also consider reaching out to someone trained to help:\n"
            "- India: AASRA Helpline — 91-9820466726\n"
            "- International: https://www.opencounseling.com/suicide-hotlines\n\n"
            "If you feel able to, would you like to tell me what’s been weighing on you?"
        ),
        "emotion": "sad",
        "confidence": 1.0,
        "sources": []
    }


In [18]:
# ========== 18+ CONTENT RESPONSE ==========

def adult_content_response():
    return {
        "text": (
            "I can’t help with explicit or pornographic content. "
            "If you have a question related to relationships, health, or general well-being, "
            "I’m happy to help in a respectful way."
        ),
        "emotion": "neutral",
        "confidence": 1.0,
        "sources": []
    }


In [19]:
# ========== SAFETY ROUTER ==========

def safety_router(user_text: str):
    content_type = classify_content(user_text)

    if content_type == "suicidal":
        return suicide_safe_response()

    if content_type == "explicit_18_plus":
        return adult_content_response()

    return None  # safe to continue to LLM


In [20]:
# ========== FINAL REPLY PIPELINE ==========

def reply(
    user_text: str,
    user_id: str,
    role: str = "assistant",
    stream: bool = False
):
    """
    Main response pipeline:
    Safety → Memory → LLM → Emotion → Store → JSON output
    """

    # 1️⃣ SAFETY OVERRIDE (18+ / suicide)
    safe_override = safety_router(user_text)
    if safe_override:
        return safe_override

    # 2️⃣ BUILD MEMORY CONTEXT (Chroma-based)
    memory_context = build_memory_context(user_id, user_text)

    # 3️⃣ CONSTRUCT PROMPT
    messages = [
        {"role": "system", "content": system_prompt()},
        {"role": "system", "content": role_prompt(role)},
    ]

    if memory_context:
        messages.append(
            {"role": "system", "content": memory_context}
        )

    messages.append(
        {"role": "user", "content": user_text}
    )

    # 4️⃣ CALL GROQ LLM
    if not stream:
        response = call_llm(messages, stream=False)
        ai_text = response.choices[0].message.content
    else:
        # Streaming generator (optional)
        def stream_generator():
            stream_resp = call_llm(messages, stream=True)
            full_text = ""
            for chunk in stream_resp:
                token = chunk.choices[0].delta.content or ""
                full_text += token
                yield token
            store_message(user_id, user_text)
        return stream_generator()

    # 5️⃣ EMOTION DETECTION
    emotion = detect_emotion(user_text)

    # 6️⃣ STORE USER MESSAGE IN MEMORY
    store_message(user_id, user_text)

    # 7️⃣ FINAL RESPONSE OBJECT
    return {
        "text": ai_text,
        "emotion": emotion,
        "confidence": 0.9,
        "sources": []
    }


In [21]:
# ========== TERMINAL / CLI CHAT LOOP ==========

def start_cli_chat(user_id="terminal_user", role="assistant"):
    """
    Interactive terminal-style chat loop.
    Type 'exit' to stop.
    """
    print("\n🧠 MIRAGE AI (Terminal Mode)")
    print("Type your message and press Enter.")
    print("Type 'exit' to quit.\n")

    while True:
        try:
            user_text = input("You: ")

            if user_text.lower() in ["exit", "quit"]:
                print("👋 Exiting MIRAGE. Goodbye!")
                break

            response = reply(
                user_text=user_text,
                user_id=user_id,
                role=role
            )

            print("\nMIRAGE:", response["text"])
            print("Emotion:", response["emotion"])
            print("-" * 50)

        except KeyboardInterrupt:
            print("\n👋 Chat interrupted. Exiting.")
            break


In [ ]:
start_cli_chat()


🧠 MIRAGE AI (Terminal Mode)
Type your message and press Enter.
Type 'exit' to quit.

